# Reproduce Table 4 (NASA Jet Engine Case Study)

We initially ran only 3 replications, so this experiment is split into 2 SLURM submissions.
These are specified by table41.yaml and table42.yaml.
They have also been collected into a single config (table_4.yaml) if you wish to reproduce all of the experiments at once.

To reproduce Table 4 from the raw results of the runs, follow these steps:

1. In the analysis directory, run: `python process_results.py table41` (and then the same for `table42`)
2. Again in the analysis directory: `python gen_combined_tables.py table41 table42`
3. Finally, run this notebook and the tables will be displayed.
4. If you instead recreate the raw results using the `table_4.yaml` config, replace the `config_names` list below with `config_names = ['table_4']`

In [1]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
from data.load import load_data
from utils.subgroup import fit_cox, in_region
from scipy.stats import sem
from IPython.display import display

## Compute Table 4 Metrics

This part computes the metric values (EPE, C-index, precision, and recall) in Table 4.

In [2]:
config_name = ['table41', 'table42']
size_threshold = '0.10'

col_order = ['base', 'random', 'prim', 'survival_tree', 'cox_tree', 'pl_ddgroup', 'c_ind_ddgroup', 'no_exp_ddgroup', 'ddgroup']
col_map = {
        'base': 'Base',
        'random': 'Rand',
        'prim': 'PRIM',
        'survival_tree': 'ST',
        'cox_tree': 'CT',
        'pl_ddgroup': 'DG-PL',
        'c_ind_ddgroup': 'DG-CI',
        'no_exp_ddgroup': 'DG-NE',
        'ddgroup': 'DG',
}
rows = ['Test EPE', 'Test C-Index', 'Test Precision', 'Test Recall']

dir_name = 'combined'
for c in config_name:
        dir_name += f'_{c}'

df = pd.read_pickle(f"../results/{dir_name}/tables/size_0.10/nasa-['Nc', 'NRf']-['op_setting_1'].pkl")
df = df[col_order]
df = df.rename(columns=col_map)

display(df.loc[rows])

,Base,Rand,PRIM,ST,CT,DG-PL,DG-CI,DG-NE,DG
Metric,,,,,,,,,
Test EPE,0.651 (0.003),0.525 (0.011),0.651 (0.003),0.532 (0.013),0.521 (0.006),0.638 (0.012),0.536 (0.010),0.509 (0.016),0.527 (0.010)
Test C-Index,0.603 (0.004),0.735 (0.010),0.602 (0.004),0.729 (0.013),0.736 (0.006),0.616 (0.013),0.728 (0.010),0.750 (0.014),0.733 (0.010)
Test Precision,0.632 (0.005),1.000 (0.000),0.640 (0.007),1.000 (0.000),1.000 (0.000),0.672 (0.037),0.999 (0.001),1.000 (0.000),1.000 (0.000)
Test Recall,1.000 (0.000),0.236 (0.024),0.911 (0.056),0.399 (0.061),0.734 (0.099),0.919 (0.081),0.290 (0.040),0.163 (0.008),0.362 (0.083)


## Compute Model Coefficients

This part computes the values of $\beta_{\mathrm{Nc}}$ and $\beta_{\mathrm{NRf}}$.

In [3]:
def select_best_regions(df, size_threshold, selection_metric='train_epe'):
    """
    For each (method, seed), select the best region based on selection_metric.
    """
    selected_rows = []

    for (method, seed), group in df.groupby(['method', 'seed']):
        # Filter by size threshold
        filtered = group[group['train_size'] >= size_threshold]

        if len(filtered) == 0:
            continue

        # Select best region (lowest selection_metric)
        best_idx = filtered[selection_metric].idxmin()
        best_row = filtered.loc[best_idx].copy()

        selected_rows.append(best_row)

    return pd.DataFrame(selected_rows)

In [4]:
indiv_dfs = []
for c in config_name:
    indiv_dfs.append(pd.read_pickle(f"../results/{c}/processed/nasa-['Nc', 'NRf']-['op_setting_1'].pkl"))

df = pd.concat(indiv_dfs, ignore_index=True)

best_regions = select_best_regions(df, 0.1)

R_star = np.array([[-10.], [0.]])

adjust_cols = [6, 7]  # Nc, NRf
subgp_cols = [0]  # op_setting_1

seeds = range(10)
region1_betas = []  # R_star region
region2_betas = []  # Complement region

for seed in seeds:
    # Load data
    X_adjust, X_subgp, Y, X_adjust_test, X_subgp_test, Y_test, B_subgp, scaler = load_data(
        'nasa', adjust_cols, subgp_cols, seed, n=5000
    )
    
    # Identify samples in each region
    in_region1 = in_region(X_subgp, R_star)
    in_region2 = ~in_region1
    
    # Fit Cox model to region 1
    X_region1 = X_adjust[in_region1]
    Y_region1 = Y[in_region1]
    if len(X_region1) > 0:
        beta1 = fit_cox(X_region1, Y_region1)
        region1_betas.append(beta1)
    
    # Fit Cox model to region 2
    X_region2 = X_adjust[in_region2]
    Y_region2 = Y[in_region2]
    if len(X_region2) > 0:
        beta2 = fit_cox(X_region2, Y_region2)
        region2_betas.append(beta2)

# Compute averages
avg_beta_region1 = np.mean(region1_betas, axis=0)
beta_r1_sem = sem(region1_betas, axis=0)
avg_beta_region2 = np.mean(region2_betas, axis=0)
beta_r2_sem = sem(region2_betas, axis=0)

print("Average Cox coefficients for Region 1 (op_setting_1 in [-10, 0]):")
print(f"  Nc:  {avg_beta_region1[0]:.2f} ({beta_r1_sem[0]:.2f})")
print(f"  NRf: {avg_beta_region1[1]:.2f} ({beta_r1_sem[1]:.2f})")
print()
print("Average Cox coefficients for Region 2 (op_setting_1 in [0, +inf]):")
print(f"  Nc:  {avg_beta_region2[0]:.2f} ({beta_r2_sem[0]:.2f})")
print(f"  NRf: {avg_beta_region2[1]:.2f} ({beta_r2_sem[1]:.2f})")

/Users/zach/anaconda3/envs/survival/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/zach/anaconda3/envs/survival/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/zach/anaconda3/envs/survival/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/zach/anaconda3/envs/survival/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and wil

Average Cox coefficients for Region 1 (op_setting_1 in [-10, 0]):
  Nc:  1.41 (0.06)
  NRf: -0.35 (0.04)

Average Cox coefficients for Region 2 (op_setting_1 in [0, +inf]):
  Nc:  2.03 (0.03)
  NRf: -0.98 (0.02)


### Results

Outputs are displayed in the form:

`<method_name> [<avg. beta_Nc value> <avg. beta_NRf value>] [<standard error of beta_Nc value> <standard error of beta_NRf value>]`

In [5]:
for method, group in best_regions.groupby('method'):
    print(method, np.round(np.mean(group['beta']), 2), np.round(sem(group['beta']), 2))

base [0.31 0.3 ] [0.01 0.01]
c_ind_ddgroup [ 2.17 -1.06] [0.09 0.06]
cox_tree [ 2.19 -1.08] [0.08 0.05]
ddgroup [ 2.23 -1.1 ] [0.09 0.06]
no_exp_ddgroup [ 2.51 -1.28] [0.05 0.04]
pl_ddgroup [0.53 0.15] [0.22 0.15]
prim [0.32 0.3 ] [0.01 0.01]
random [ 2.35 -1.19] [0.1  0.07]
survival_tree [ 2.   -0.94] [0.09 0.1 ]
